### Read and replace webpage
Ongoing notebook to read webpage data, perform LS, generate replacement webpage with LS data

In [29]:
###### import packages, sentence model, and parse html from site url
import requests
import re
from bs4 import BeautifulSoup
import string
from utils.TS_pipeline import TS_with_BART
url = 'https://www.bbc.com/zhongwen/articles/c4gl97d2rzjo/simp'
site = requests.get(url)
site_soup = BeautifulSoup(site.text, 'html.parser')
# print(site_soup.prettify())

In [30]:
def zng(paragraph: str):
    for sent in re.findall(r'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph):
        yield sent

for element in site_soup.find_all('p'):  # Get all text nodes
    html = element.decode_contents()  # Preserve inner HTML
    element.clear()  # Remove existing content
    sentences = list(zng(html))
    new_html = ''
    for sentence in sentences:
        new_html += TS_with_BART(sentence)
    # text = text.replace(old_word, f'<span style="color: red;">{new_word}</span>')
    # newtext = BeautifulSoup(convert, 'html.parser')
    # element.replace_with(newtext)
   
    element.append(BeautifulSoup(new_html, 'html.parser'))

C:\Users\tempu\AppData\Local\Temp\ipykernel_12924\1092285010.py:16: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  element.append(BeautifulSoup(new_html, 'html.parser'))


In [31]:
with open("output.html", "w", encoding = 'utf-8') as file: 
    # prettify the soup object and convert it into a string 
    file.write(str(site_soup)) 